In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, MetaData, text, update
from smartsheet_dataframe import get_as_df

In [2]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
mssid = 3005704744265604 # The ID number that references the MSS sheet

In [3]:
mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid,
                  )

mss_df.head()

,row_id,parent_id,IntakeID,FolderID,UniqueID,TicketID,Tract,CMP,Stand,Acres,...,Activity Implemented,Applied GA_ACRES,PstBA,PstTPA,PstGTA,RP Completed,Notes,Completed Date,Modified,Modified By
0,6828474529845124,,WAR240426,WAR240426,WAR240426CC16,WAR240426CC16,Melson,,7,36.62,...,,,,,,,Acres in pre-imp excel sheet did not match acr...,,2024-05-02T15:52:54Z,
1,1921938250633092,,WAR240426,WAR240426,WAR240426CC15,WAR240426CC15,Melson,,7,21.72,...,,,,,,,Acres in pre-imp excel sheet did not match acr...,,2024-05-02T15:52:54Z,
2,5042111732092804,,WAR240426,WAR240426,WAR240426CC14,WAR240426CC14,Melson,,7,38.0,...,,,,,,,Acres in pre-imp excel sheet did not match acr...,,2024-05-02T15:52:54Z,
3,3097092842426244,,WAR240426,WAR240426,WAR240426CC13,WAR240426CC13,Melson,,7,34.65,...,,,,,,,Acres in pre-imp excel sheet did not match acr...,,2024-05-02T15:52:54Z,
4,7241169347383172,,WAR240426,WAR240426,WAR240426CC12,WAR240426CC12,Melson,,7,30.72,...,,,,,,,Acres in pre-imp excel sheet did not match acr...,,2024-05-02T15:52:54Z,


In [4]:
ticket_id_df = mss_df.groupby(['TicketID']).agg({'UniqueID': list}).reset_index()

In [5]:
ticket_id_df.loc[ticket_id_df['TicketID'].str.contains("XX")][:5]

,TicketID,UniqueID
10,BJT190424THXX,"[BJT190424AH01, BJT190424AH02, BJT190424TH01, ..."
19,BJT210628XXXX,"[BJT210628TH01, BJT210628TH02]"
24,BJT221103THXX,[BJT221103TH02]
32,BKG220831THXX,"[BKG220831TH01, BKG220831TH02, BKG220831TH03, ..."
40,DLT220623THXX,"[DLT220623TH04, DLT220623TH08]"


In [6]:
ticket_ids = mss_df.loc[mss_df['TicketID'].str.contains("XX"), 'TicketID'].unique().tolist()

In [7]:
len(ticket_ids)

34

In [8]:
ticket_id_df = pd.DataFrame(ticket_id_df['TicketID'])

In [9]:
ticket_id_df

,TicketID
0,BJT010101TH02
1,BJT010101TH05
2,BJT171024TH01
3,BJT171024TH07
4,BJT171024TT01
...,...
489,WRR240318BN01
490,WRR240318BN02
491,WRR240325BN01
492,WRR240404TH01


In [10]:
ticket_id_df.rename(columns={'TicketID': 'ticketid'}, inplace=True)

In [11]:
ticket_id_df

,ticketid
0,BJT010101TH02
1,BJT010101TH05
2,BJT171024TH01
3,BJT171024TH07
4,BJT171024TT01
...,...
489,WRR240318BN01
490,WRR240318BN02
491,WRR240325BN01
492,WRR240404TH01


In [12]:
# setup db connection
db_host = 'localhost'
db_name = 'HarvestManagementTrial'
db_user = 'postgres'
db_password = 'root'

connection_string = f'postgresql://{db_user}:{db_password}@{db_host}/{db_name}'

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

ticket_id_df.to_sql('ticketid', engine, if_exists='append', index=False)

## Alright. So the TicketID table is made.

Next Step:
- get the ticketid and ticketid_id
- make a dataframe that has a column of unique id and their associated ticketid_id
- Populate the ticketid field in the client intake table with the ticketid_id


In [13]:
ticket_id_query = "SELECT * FROM ticketid"
ticket_id_data = pd.read_sql(ticket_id_query, engine)

In [14]:
ticket_id_data

,ticketid_id,ticketid
0,1,BJT010101TH02
1,2,BJT010101TH05
2,3,BJT171024TH01
3,4,BJT171024TH07
4,5,BJT171024TT01
...,...,...
489,490,WRR240318BN01
490,491,WRR240318BN02
491,492,WRR240325BN01
492,493,WRR240404TH01


In [15]:
client_datatable = mss_df[['UniqueID', 'TicketID']]

In [16]:
client_datatable

,UniqueID,TicketID
0,WAR240426CC16,WAR240426CC16
1,WAR240426CC15,WAR240426CC15
2,WAR240426CC14,WAR240426CC14
3,WAR240426CC13,WAR240426CC13
4,WAR240426CC12,WAR240426CC12
...,...,...
600,WRR230829CC04,WRR230829CC04
601,WRR230829CC03,WRR230829CCXX
602,WRR230829CC02,WRR230829CCXX
603,WRR230829CC01,WRR230829CCXX


In [17]:
joined_table = client_datatable.merge(ticket_id_data,how='left', left_on='TicketID', right_on='ticketid')

In [18]:
joined_table

,UniqueID,TicketID,ticketid_id,ticketid
0,WAR240426CC16,WAR240426CC16,296,WAR240426CC16
1,WAR240426CC15,WAR240426CC15,295,WAR240426CC15
2,WAR240426CC14,WAR240426CC14,294,WAR240426CC14
3,WAR240426CC13,WAR240426CC13,293,WAR240426CC13
4,WAR240426CC12,WAR240426CC12,292,WAR240426CC12
...,...,...,...,...
600,WRR230829CC04,WRR230829CC04,456,WRR230829CC04
601,WRR230829CC03,WRR230829CCXX,459,WRR230829CCXX
602,WRR230829CC02,WRR230829CCXX,459,WRR230829CCXX
603,WRR230829CC01,WRR230829CCXX,459,WRR230829CCXX


In [19]:
result_table = joined_table[['UniqueID', 'ticketid_id']]
result_table.head()

,UniqueID,ticketid_id
0,WAR240426CC16,296
1,WAR240426CC15,295
2,WAR240426CC14,294
3,WAR240426CC13,293
4,WAR240426CC12,292


In [20]:
table_name = 'client_table_preimp'
column_to_update = 'ticketid'

In [21]:
conn = engine.connect()


for index, row in result_table.iterrows():
    unique_id = row['UniqueID']
    new_value = row['ticketid_id']
    sql_query = text(f"UPDATE {table_name} SET {column_to_update} = :new_value WHERE client_table_preimp.uniqueid = :unique_id")
    
    result = conn.execute(sql_query, {"new_value": new_value, "unique_id": unique_id})
    
    if result.rowcount == 0:
        print(f"No rows updated for unique_id {unique_id}")




No rows updated for unique_id WAR240426CC16
No rows updated for unique_id WAR240426CC15
No rows updated for unique_id WAR240426CC14
No rows updated for unique_id WAR240426CC13
No rows updated for unique_id WAR240426CC12
No rows updated for unique_id WAR240426CC11
No rows updated for unique_id WAR240426CC10
No rows updated for unique_id WAR240426CC09
No rows updated for unique_id WAR240426CC08
No rows updated for unique_id WAR240426CC07
No rows updated for unique_id WAR240426CC06
No rows updated for unique_id WAR240426CC05
No rows updated for unique_id WAR240426CC04
No rows updated for unique_id WAR240426CC03
No rows updated for unique_id WAR240426CC02
No rows updated for unique_id WAR240426CC01
No rows updated for unique_id WAR240429BI01
No rows updated for unique_id WRR210305MW01
No rows updated for unique_id WRR210305MW02
No rows updated for unique_id WAR240417TH01
No rows updated for unique_id WAR240426TH01
No rows updated for unique_id WRR230220HD01
No rows updated for unique_id WA

In [22]:
conn.commit()

In [23]:
conn.close()

In [24]:
result_table[result_table['UniqueID'] == 'WRR240404TH02']

,UniqueID,ticketid_id
79,WRR240404TH02,494
